In [ ]:
!pip install transformers datasets torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import Dataset

# Load your CSV file
df = pd.read_csv('/content/Complete_Chatbot_Data.csv')

# Convert to a format compatible with BART (question as input, answer as output)
def preprocess_data(df):
    conversations = []
    for _, row in df.iterrows():
        conversations.append({
            'input_text': row['question'],
            'target_text': row['answer']
        })
    return conversations

# Convert to Hugging Face Dataset
data = preprocess_data(df)
dataset = Dataset.from_pandas(pd.DataFrame(data))


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model_name = "facebook/bart-large"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], max_length=512, truncation=True, padding='max_length')
    targets = tokenizer(examples['target_text'], max_length=128, truncation=True, padding='max_length')
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1855 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,      # Log every 10 steps
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # You might want to provide a separate validation dataset
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,0.275400,0.224470
2,0.198800,0.155665
3,0.151800,0.137459


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=1392, training_loss=0.9243037287659687, metrics={'train_runtime': 1972.5391, 'train_samples_per_second': 2.821, 'train_steps_per_second': 0.706, 'total_flos': 6029968555376640.0, 'train_loss': 0.9243037287659687, 'epoch': 3.0})

In [ ]:
import torch

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# def chat_with_bot():
#     while True:
#         user_input = input("You: ")
#         input_ids = tokenizer.encode(user_input, return_tensors="pt").to(device)
#         output_ids = model.generate(input_ids, max_length=200, num_beams=4, early_stopping=True)
#         bot_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#         print(f"Bot: {bot_response}")

# chat_with_bot()


In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('./fine-tuned-bart')
tokenizer.save_pretrained('./fine-tuned-bart')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart/tokenizer_config.json',
 './fine-tuned-bart/special_tokens_map.json',
 './fine-tuned-bart/vocab.json',
 './fine-tuned-bart/merges.txt',
 './fine-tuned-bart/added_tokens.json')

In [ ]:
!zip -r bart-large.zip /content/fine-tuned-bart

  adding: content/fine-tuned-bart/ (stored 0%)
  adding: content/fine-tuned-bart/config.json (deflated 64%)
  adding: content/fine-tuned-bart/special_tokens_map.json (deflated 85%)
  adding: content/fine-tuned-bart/tokenizer_config.json (deflated 76%)
  adding: content/fine-tuned-bart/generation_config.json (deflated 47%)
  adding: content/fine-tuned-bart/vocab.json (deflated 68%)
  adding: content/fine-tuned-bart/merges.txt (deflated 53%)
  adding: content/fine-tuned-bart/model.safetensors (deflated 7%)


## **Inference Code**

In [ ]:
# from transformers import BartForConditionalGeneration, BartTokenizer

# # Load the fine-tuned model and tokenizer
# model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
# tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

# # Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)


In [ ]:
# def chat_with_bot():
#     while True:
#         user_input = input("You: ")
#         input_ids = tokenizer.encode(user_input, return_tensors="pt").to(device)
#         output_ids = model.generate(input_ids, max_length=200, num_beams=4, early_stopping=True)
#         bot_response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
#         print(f"Bot: {bot_response}")

# chat_with_bot()
